# Graph
> 정점(vertex)과 간선(edge)으로 이루어진 연결 관계를 표현하는 자료구조

| 분류            | 알고리즘                                   | 핵심 목적             |
| ------------- | -------------------------------------- | ----------------- |
| **최단 경로**     | Dijkstra, Bellman-Ford, Floyd-Warshall | 노드 간 최단 거리 계산     |
| **최소 신장 트리**  | Kruskal, Prim                          | 모든 노드를 최소 비용으로 연결 |
| **정렬형 탐색**    | Topological Sort                       | 방향 그래프의 순서 결정     |
| **그래프 집합 관리** | Union-Find                             | 노드 집합 병합/판별       |


## (1) 최단경로
### 1. dijkstra 알고리즘
> 가중치가 양수일 때, 하나의 시작점에서 모든 노드까지의 최단거리를 구하는 알고리즘

- 핵심 아이디어
  - 방문하지 않은 노드 중, 가장 가까운 노드를 선택해 탐색을 확장
  - 우선순위 큐(힙)를 사용하면 효율적 (O(E log V))

In [2]:
import heapq

def dijkstra(graph, start):
    distance = {node: float('inf') for node in graph}
    distance[start] = 0
    pq = [(0, start)]

    while pq:
        dist, node = heapq.heappop(pq)
        if dist > distance[node]:
            continue

        for nxt, weight in graph[start]:
            new_dist = dist + weight
            if new_dist < distance[nxt]:
                distance[nxt] = new_dist
                heapq.heappush(pq, (new_dist, nxt))

    
    return distance

graph = {
    'A': [('B', 8), ('C', 1)],
    'B': [('D', 2)],
    'C': [('B', 5), ('D', 9)],
    'D': []
}

print(dijkstra(graph, 'A'))

{'A': 0, 'B': 8, 'C': 1, 'D': inf}


### 2. bellman-ford 알고리즘
> 음수 가중치가 포함된 그래프에서도 최단 경로 계산 가능
> 
> 단, 음수 사이클이 존재하면 탐색 불가능

- 핵심 아이디어
  - 모든 간선을 v-1번 반복하면 최단 경로 계산이 가능하다라는 배경으로 
  - 마지막에 한번 더 반복하여 값이 갱신되면 -> 음수 사이클이 존재
- 복잡도는 O(VE) 로 다익스트라보다 느리다


In [3]:
def bellman_foard(n, edges, start):
    distance = [float('inf')] * (n+1)
    distance[start] = 0

    for _ in range(n-1):
        for u, v, w in edges:
            if distance[u] != float('inf') and distance[u] + w < distance[v]:
                distance[v] = distance[u] + w

    
    # 음수 사이클 확인
    for u, v, w in edges:
        if distance[u] != float('inf') and distance[u] + w < distance[v]:
            return "negative cycle detected"
        
    
    return distance

### 3. floyd-warshall 알고리즘
> 모든 노드 쌍 간 최단 거리 계산
> 
> dp 기반의 전형적인 3중 루프 알고리즘

- 핵심 아이디어
  - 각 노드 k를 거쳐 가는 경우의 최소값을 업데이트
  - dist[i][j] = min(dist[i][j], dist[i][k] + dist[k][j])
- 복잡도는 O(n^3)

In [4]:
def floyd_warshall(graph):
    n = len(graph)
    distance = [row[:] for row in graph]

    for k in range(n):
        for i in range(n):
            for j in range(n):
                distance[i][j] = min(distance[i][j], distance[i][k] + distance[k][j])
    
    return distance

## (2) 최소 신장 트리 (MST, minimum spanning tree)
> 그래프의 모든 정점을 최소 비용으로 연결하는 트리
>
> 사이클 x, 모든 노드 연결

### 1. kruskal 알고리즘
> 간선을 오름차순 정렬 -> 가장 짧은 간선부터 선택
>
> 단, 사이클이 생기지 않게 union-find로 연결 관리

- 핵심 아이디어
  - 간선을 가중치 기준으로 정렬
  - 사이클이 생기지 않으면 추가
  - 모든 노드가 연결될 때까지 반복

In [5]:
def find(parent, x):
    if parent[x] != x:
        parent[x] = find(parent, parent[x])
    return parent[x]

def union(parent, a, b):
    a = find(parent, a)
    b = find(parent, b)
    if a < b:
        parent[b] = a
    else:
        parent[a] = b

def kruskal(n, edges):
    parent = [i for i in range(n+1)]
    edges.sort(key=lambda x: x[2])
    total = 0

    for a, b, cost in edges:
        if find(parent, a) != find(parent, b):
            union(parent, a, b)
            total += cost

    return total

### 2. Prime 알고리즘
> 한 정점에서 시작해서, 가장 가까운 노드를 하나씩 추가
>
> dijkstra와 매우 유사 (힙 기반)

In [6]:
import heapq

def prim(graph, start):
    visited= set()
    pq = [(0, start)]
    total = 0

    while pq:
        cost, node = heapq.heappop(pq)
        if node in visited:
            continue
        visited.add(node)
        total += cost

        for nxt, weight in graph[node]:
            if nxt not in visited:
                heapq.heappush(pq, (weight, nxt))

    return total

## (3) 위상 정렬
> 방향 그래프(DAG)에서 순서가 있는 작업 순서를 결정하는 알고리즘

- 핵심 아이디어
  - 진입 차수가 0인 노드를 큐에 삽입
  - 큐에서 하나 꺼내 인접 노트의 진입 차수를 감소
  - 새로 진입 차수가 0이 되면 큐에 삽입
  - 큐가 빌때까지 반복

In [7]:
from collections import deque

def topological_sort(n, edges):
    indegree = [0] * (n+1)
    graph = [[] for _ in range(n+1)]

    for u, v in edges:
        graph[u].append(v)
        indegree[v] += 1

    q = deque([i for i in range (1, n+1) if indegree[i] == 0])
    result = []

    while q:
        node = q.popleft()
        result.append(node)

        for nxt in graph[node]:
            indegree[nxt] -= 1
            if indegree[nxt] == 0:
                q.append(nxt)
    
    return result

---

## 문제1. 가장 먼 노드
n개의 노드가 있는 그래프가 있습니다. 각 노드는 1부터 n까지 번호가 적혀있습니다. 1번 노드에서 가장 멀리 떨어진 노드의 갯수를 구하려고 합니다. 가장 멀리 떨어진 노드란 최단경로로 이동했을 때 간선의 개수가 가장 많은 노드들을 의미합니다.

노드의 개수 n, 간선에 대한 정보가 담긴 2차원 배열 vertex가 매개변수로 주어질 때, 1번 노드로부터 가장 멀리 떨어진 노드가 몇 개인지를 return 하도록 solution 함수를 작성해주세요.


In [15]:
from collections import deque

def solution(n, edge):
    graph = [[] for _ in range(n+1)]
    
    for u, v in edge:
        graph[u].append(v)
        graph[v].append(u)

    visited = [False] * (n+1)
    distance = [0] * (n+1)
    pq = deque([1])
    visited[1] = True

    while pq:
        node = pq.popleft()
        for nxt in graph[node]:
            if not visited[nxt]:
                visited[nxt] = True
                distance[nxt] = distance[node] + 1
                pq.append(nxt)

    max_dist = max(distance)
    
    return distance.count(max_dist)


n = 6
vertex = [[3, 6], [4, 3], [3, 2], [1, 3], [1, 2], [2, 4], [5, 2]]
print(solution(n, vertex))

3


---

## 문제2. 순위
n명의 권투선수가 권투 대회에 참여했고 각각 1번부터 n번까지 번호를 받았습니다. 권투 경기는 1대1 방식으로 진행이 되고, 만약 A 선수가 B 선수보다 실력이 좋다면 A 선수는 B 선수를 항상 이깁니다. 심판은 주어진 경기 결과를 가지고 선수들의 순위를 매기려 합니다. 하지만 몇몇 경기 결과를 분실하여 정확하게 순위를 매길 수 없습니다.

선수의 수 n, 경기 결과를 담은 2차원 배열 results가 매개변수로 주어질 때 정확하게 순위를 매길 수 있는 선수의 수를 return 하도록 solution 함수를 작성해주세요.

In [ ]:
def solution(n, results):
    
    graph = [[0] * (n+1) for _ in range(n+1)]

    # 0 : 모름 1 : i가 j를 이김 -1 : i가 j에게 짐
    for a, b in results:
        graph[a][b] = 1 
        graph[b][a] = -1

    for k in range(1, n+1):
        for i in range(1, n+1):
            for j in range(1, n+1):
                if graph[i][k] == 1 and graph[k][j] == 1:
                    graph[i][j] = 1
                    graph[j][i] = -1

    answer = 0
    for i in range(1, n+1):
        known = 0 # i가 다른 선수들과의 관계를 알고 있는 횟수
        for j in range(1, n+1):
            if graph[i][j] != 0: # i와 j의 승패관계가 확실하다면
                known += 1
        
        if known == n-1: # 자기 자신을 제외한 모든 선수와의 관계가 명확함
            answer += 1
            

    return answer


n = 5
results = [[4, 3], [4, 2], [3, 2], [1, 2], [2, 5]]
print(solution(n, results))

2


위상정렬로 풀어야하나 처음에 생각이 들 수 있지만 이 문제는 각 선수의 상대적 순위가 확실히 결정된 선수의 수를 구해야하는 문제다. 그래서 플로이드 워셜 알고리즘을 통해 모든 선수 쌍 간의 승패 관계를 계산해야 한다. 그렇게 관계를 유추한 후에 어떤 선수 i에 대해 자신을 제외한 n-1 명의 승패가 모두 확정되어 있으면 그 선수의 순위를 알 수 있다!
